In [ ]:

# use natural language toolkit
import nltk

from nltk.stem import PorterStemmer
# word stemmer
stemmer = PorterStemmer()

In [80]:
# 3 classes of training data
training_data = []
training_data.append({"class":"greeting", "sentence":"how are you?"})
training_data.append({"class":"greeting", "sentence":"how is your day?"})
training_data.append({"class":"greeting", "sentence":"good day"})
training_data.append({"class":"greeting", "sentence":"how is it going today?"})

training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"see you later"})
training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"talk to you soon"})

training_data.append({"class":"sandwich", "sentence":"make me a sandwich"})
training_data.append({"class":"sandwich", "sentence":"can you make a sandwich?"})
training_data.append({"class":"sandwich", "sentence":"having a sandwich today?"})
training_data.append({"class":"sandwich", "sentence":"what's for lunch?"})
print ("%s sentences of training data" % len(training_data))

12 sentences of training data


In [81]:
# capture unique stemmed words in the training corpus
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
print("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
print("Class words: %s" % class_words)

In [82]:
# we can now calculate a score for a new sentence
sentence = "good day for us to have lunch?"

# calculate a score for a given class
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with same weight
            score += 1
            
            if show_details:
                print("   match: %s" % stemmer.stem(word.lower() ))
    return score

In [83]:
# now we can find the class with the highest score
for c in class_words.keys():
    print("Class: %s  Score: %s \n" % (c, calculate_class_score(sentence, c)))

   match: for
   match: have
   match: lunch
Class: sandwich  Score: 3 

   match: good
   match: day
Class: greeting  Score: 2 

   match: day
   match: to
   match: have
Class: goodbye  Score: 3 



In [84]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score_commonality(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative weight
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [85]:
# now we can find the class with the highest score
for c in class_words.keys():
    print("Class: %s  Score: %s \n" % (c, calculate_class_score_commonality(sentence, c)))

   match: for (1.0)
   match: have (0.3333333333333333)
   match: lunch (1.0)
Class: sandwich  Score: 2.333333333333333 

   match: good (1.0)
   match: day (0.25)
Class: greeting  Score: 1.25 

   match: day (0.25)
   match: to (1.0)
   match: have (0.3333333333333333)
Class: goodbye  Score: 1.5833333333333333 



In [86]:
# return the class with highest score for sentence
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score_commonality(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score

In [87]:
classify("make me some lunch?")

('sandwich', 2.5)

In [88]:
classify("who are you?")

('greeting', 1.25)

In [89]:
classify("am I crazy?")

(None, 0)